# **Train RF-DETR-N (Nano)**

RF-DETRは、Roboflowが開発したリアルタイムの物体検出モデルです。
このノートブックでは、RF-DETR-N（Nano）モデルをカスタムデータセットでトレーニングします。

参考: https://github.com/roboflow/rf-detr

---

## ⚠️ 使用前の注意

**データの前処理は `train_yolo-seg.ipynb` で行ってください。**

このノートブックは、`train_yolo-seg.ipynb` で準備されたYOLO形式のデータ（`data/train/`）を
COCO形式に変換してRF-DETRでトレーニングします。

### ワークフロー
1. **`train_yolo-seg.ipynb`** でデータの前処理・分割を実行
2. **このノートブック** でYOLO→COCO変換 & RF-DETR-Nトレーニングを実行

※ `data/train/` 内のデータは毎回のトレーニングで更新されます。


In [1]:
# CUDAが使えるかどうかを確認
import torch

cuda_available = torch.cuda.is_available()

if cuda_available:
    print(f"CUDA is available! GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")


CUDA is available! GPU: Quadro RTX 5000


In [2]:
# 環境の確認
import torch
import sys
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"PyTorch CUDA version: {torch.version.cuda}")


Python version: 3.8.6 (tags/v3.8.6:db45529, Sep 23 2020, 15:52:53) [MSC v.1927 64 bit (AMD64)]
PyTorch version: 2.4.1+cu121
CUDA available: True
PyTorch CUDA version: 12.1


In [ ]:
# RF-DETRのインストール
# 初回のみ実行してください
# !pip install rfdetr


## **RF-DETR データセット構造**

RF-DETRはCOCO形式のデータセットを使用します。

```
dataset/
├── train/
│   ├── image1.jpg
│   ├── image2.jpg
│   └── _annotations.coco.json
├── valid/
│   ├── image1.jpg
│   └── _annotations.coco.json
```

既存のYOLO形式データをCOCO形式に変換する必要があります。


## **Step 1: データの準備**

train_yolo-seg.ipynbと同様のデータ前処理を行います。


In [4]:
# データディレクトリの設定
import os

# 既存のYOLO形式データのパス
YOLO_DATA_DIR = r'C:\Users\ykita\ROP_AI_project\ROP_project\data\train'

# RF-DETR用のCOCO形式データの出力パス
COCO_DATA_DIR = r'C:\Users\ykita\ROP_AI_project\ROP_project\data\train_coco'

# クラス定義
CLASSES = ['Fundus', 'Disc', 'Macula']
NUM_CLASSES = len(CLASSES)

print(f"YOLO Data Directory: {YOLO_DATA_DIR}")
print(f"COCO Data Directory: {COCO_DATA_DIR}")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Classes: {CLASSES}")


YOLO Data Directory: C:\Users\ykita\ROP_AI_project\ROP_project\data\train
COCO Data Directory: C:\Users\ykita\ROP_AI_project\ROP_project\data\train_coco
Number of classes: 3
Classes: ['Fundus', 'Disc', 'Macula']


In [5]:
# YOLO形式（セグメンテーション）からCOCO形式への変換
#
# YOLO Segmentation形式:
#   class_id x1 y1 x2 y2 x3 y3 ... (normalized coordinates)
#
# COCO Detection形式:
#   annotations.json with bounding boxes

import json
import os
from PIL import Image
from tqdm import tqdm
import shutil

def yolo_seg_to_coco(yolo_images_dir, yolo_labels_dir, output_dir, classes, split_name):
    """
    YOLO Segmentation形式からCOCO Detection形式に変換
    
    Args:
        yolo_images_dir: YOLO形式の画像ディレクトリ
        yolo_labels_dir: YOLO形式のラベルディレクトリ
        output_dir: 出力ディレクトリ
        classes: クラスのリスト
        split_name: 'train' or 'valid'
    """
    # 出力ディレクトリの作成
    output_images_dir = os.path.join(output_dir, split_name)
    os.makedirs(output_images_dir, exist_ok=True)
    
    # COCO形式のアノテーション構造
    coco_annotations = {
        "images": [],
        "annotations": [],
        "categories": []
    }
    
    # カテゴリの追加
    for idx, class_name in enumerate(classes):
        coco_annotations["categories"].append({
            "id": idx,
            "name": class_name,
            "supercategory": "object"
        })
    
    annotation_id = 0
    image_id = 0
    
    # 画像ファイルのリストを取得
    image_files = [f for f in os.listdir(yolo_images_dir) 
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    print(f"Processing {len(image_files)} images for {split_name}...")
    
    for img_file in tqdm(image_files, desc=f"Converting {split_name}"):
        img_path = os.path.join(yolo_images_dir, img_file)
        label_file = os.path.splitext(img_file)[0] + '.txt'
        label_path = os.path.join(yolo_labels_dir, label_file)
        
        # 画像サイズの取得
        try:
            with Image.open(img_path) as img:
                width, height = img.size
        except Exception as e:
            print(f"Error reading image {img_path}: {e}")
            continue
        
        # 画像情報の追加
        coco_annotations["images"].append({
            "id": image_id,
            "file_name": img_file,
            "width": width,
            "height": height
        })
        
        # 画像をコピー
        dst_img_path = os.path.join(output_images_dir, img_file)
        if not os.path.exists(dst_img_path):
            shutil.copy(img_path, dst_img_path)
        
        # ラベルファイルの処理
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()
            
            for line in lines:
                parts = line.strip().split()
                if len(parts) < 5:  # 少なくともclass_id + 2点（4座標）が必要
                    continue
                
                class_id = int(parts[0])
                
                # セグメンテーション座標を取得
                coords = list(map(float, parts[1:]))
                
                # ペアになっていない場合はスキップ
                if len(coords) % 2 != 0:
                    continue
                
                # 正規化された座標をピクセル座標に変換
                x_coords = [coords[i] * width for i in range(0, len(coords), 2)]
                y_coords = [coords[i] * height for i in range(1, len(coords), 2)]
                
                # バウンディングボックスを計算
                x_min = min(x_coords)
                y_min = min(y_coords)
                x_max = max(x_coords)
                y_max = max(y_coords)
                
                bbox_width = x_max - x_min
                bbox_height = y_max - y_min
                
                # セグメンテーション（ポリゴン）の作成
                segmentation = []
                for i in range(0, len(coords), 2):
                    segmentation.append(coords[i] * width)
                    segmentation.append(coords[i + 1] * height)
                
                # アノテーションの追加
                coco_annotations["annotations"].append({
                    "id": annotation_id,
                    "image_id": image_id,
                    "category_id": class_id,
                    "bbox": [x_min, y_min, bbox_width, bbox_height],
                    "area": bbox_width * bbox_height,
                    "segmentation": [segmentation],
                    "iscrowd": 0
                })
                annotation_id += 1
        
        image_id += 1
    
    # アノテーションファイルの保存
    annotations_path = os.path.join(output_images_dir, '_annotations.coco.json')
    with open(annotations_path, 'w') as f:
        json.dump(coco_annotations, f, indent=2)
    
    print(f"Saved {split_name} annotations to {annotations_path}")
    print(f"  - Images: {len(coco_annotations['images'])}")
    print(f"  - Annotations: {len(coco_annotations['annotations'])}")
    
    return coco_annotations


In [6]:
# YOLO形式からCOCO形式に変換を実行

# 出力ディレクトリのクリア（必要に応じて）
if os.path.exists(COCO_DATA_DIR):
    print(f"Removing existing directory: {COCO_DATA_DIR}")
    shutil.rmtree(COCO_DATA_DIR)
os.makedirs(COCO_DATA_DIR, exist_ok=True)

# Train データの変換
train_yolo_images = os.path.join(YOLO_DATA_DIR, 'images', 'train')
train_yolo_labels = os.path.join(YOLO_DATA_DIR, 'labels', 'train')

print("\n=== Converting Train Data ===")
train_coco = yolo_seg_to_coco(
    yolo_images_dir=train_yolo_images,
    yolo_labels_dir=train_yolo_labels,
    output_dir=COCO_DATA_DIR,
    classes=CLASSES,
    split_name='train'
)

# Valid データの変換
valid_yolo_images = os.path.join(YOLO_DATA_DIR, 'images', 'valid')
valid_yolo_labels = os.path.join(YOLO_DATA_DIR, 'labels', 'valid')

print("\n=== Converting Valid Data ===")
valid_coco = yolo_seg_to_coco(
    yolo_images_dir=valid_yolo_images,
    yolo_labels_dir=valid_yolo_labels,
    output_dir=COCO_DATA_DIR,
    classes=CLASSES,
    split_name='valid'
)

print("\n=== Conversion Complete ===")



=== Converting Train Data ===
Processing 16479 images for train...


Converting train: 100%|██████████| 16479/16479 [02:22<00:00, 116.01it/s]


Saved train annotations to C:\Users\ykita\ROP_AI_project\ROP_project\data\train_coco\train\_annotations.coco.json
  - Images: 16479
  - Annotations: 16823

=== Converting Valid Data ===
Processing 4023 images for valid...


Converting valid: 100%|██████████| 4023/4023 [00:36<00:00, 110.62it/s]


Saved valid annotations to C:\Users\ykita\ROP_AI_project\ROP_project\data\train_coco\valid\_annotations.coco.json
  - Images: 4023
  - Annotations: 4858

=== Conversion Complete ===


In [7]:
# 変換結果の確認

import os

def count_files_in_directory(directory):
    """指定されたディレクトリ内のファイル数をカウントする"""
    if not os.path.exists(directory):
        return 0
    return len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])

train_dir = os.path.join(COCO_DATA_DIR, 'train')
valid_dir = os.path.join(COCO_DATA_DIR, 'valid')

print("=== COCO Format Data Summary ===")
print(f"Train directory: {train_dir}")
print(f"  Files: {count_files_in_directory(train_dir)}")
print(f"Valid directory: {valid_dir}")
print(f"  Files: {count_files_in_directory(valid_dir)}")


=== COCO Format Data Summary ===
Train directory: C:\Users\ykita\ROP_AI_project\ROP_project\data\train_coco\train
  Files: 16480
Valid directory: C:\Users\ykita\ROP_AI_project\ROP_project\data\train_coco\valid
  Files: 4024


## **Step 2: RF-DETR-N トレーニング**


In [11]:
# RF-DETRのインポートとモデルの初期化
from rfdetr import RFDETRNano

# RF-DETR-Nano モデルの作成
model = RFDETRNano()

print("RF-DETR-Nano model loaded successfully!")


ImportError: cannot import name 'RFDETRNano' from 'rfdetr' (c:\Users\ykita\ROP_AI_project\ropenv\lib\site-packages\rfdetr\__init__.py)

In [ ]:
# トレーニングパラメータの設定

# トレーニング設定
EPOCHS = 100
BATCH_SIZE = 8
LEARNING_RATE = 1e-4
LEARNING_RATE_ENCODER = 1.5e-4
WEIGHT_DECAY = 1e-4
RESOLUTION = 560  # RF-DETRは56の倍数が推奨

# Early Stopping設定
EARLY_STOPPING = True
EARLY_STOPPING_PATIENCE = 20

# 出力ディレクトリ
OUTPUT_DIR = r'C:\Users\ykita\ROP_AI_project\ROP_project\runs\rfdetr'

print("=== Training Configuration ===")
print(f"Epochs: {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Resolution: {RESOLUTION}")
print(f"Output Directory: {OUTPUT_DIR}")


In [ ]:
# RF-DETR-N トレーニングの実行

model.train(
    dataset_dir=COCO_DATA_DIR,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    grad_accum_steps=4,
    lr=LEARNING_RATE,
    lr_encoder=LEARNING_RATE_ENCODER,
    weight_decay=WEIGHT_DECAY,
    resolution=RESOLUTION,
    use_ema=True,
    checkpoint_interval=10,
    tensorboard=True,
    early_stopping=EARLY_STOPPING,
    early_stopping_patience=EARLY_STOPPING_PATIENCE,
    early_stopping_min_delta=0.001,
    early_stopping_use_ema=True,
    device="cuda",
    output_dir=OUTPUT_DIR
)


## **Step 3: 推論テスト**


In [ ]:
# 学習済みモデルのロード
from rfdetr import RFDETRNano

# 学習済みモデルのパス（トレーニング後に生成される）
model_path = os.path.join(OUTPUT_DIR, 'best_model.pt')  # パスは実際の出力に合わせて調整

# モデルのロード
# model = RFDETRNano(pretrain_weights=model_path)
model = RFDETRNano()

print("Model loaded successfully!")


In [ ]:
# テスト画像での推論
import cv2
import matplotlib.pyplot as plt

# テスト画像のパス
test_image_path = r'C:\Users\ykita\ROP_AI_project\ROP_project\data\ROP_image\IMG_2025\IMG_2025_0016.jpg'

# 推論実行
detections = model.predict(test_image_path, threshold=0.5)

# 結果の表示
print(f"Detections: {len(detections)}")
for det in detections:
    print(f"  Class: {det['class']}, Confidence: {det['confidence']:.4f}, BBox: {det['bbox']}")


In [ ]:
# 検出結果の可視化
import cv2
import matplotlib.pyplot as plt
import numpy as np

def visualize_detections(image_path, detections, classes, threshold=0.5):
    """
    検出結果を画像に描画して表示
    """
    # 画像の読み込み
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # カラーマップ
    colors = [
        (255, 0, 0),    # Red
        (0, 255, 0),    # Green
        (0, 0, 255),    # Blue
        (255, 255, 0),  # Yellow
        (255, 0, 255),  # Magenta
    ]
    
    # 検出結果の描画
    for det in detections:
        if det['confidence'] >= threshold:
            class_id = det['class_id'] if 'class_id' in det else 0
            bbox = det['bbox']
            color = colors[class_id % len(colors)]
            
            # バウンディングボックスの描画
            x1, y1, x2, y2 = map(int, bbox)
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            
            # ラベルの描画
            label = f"{classes[class_id]}: {det['confidence']:.2f}"
            cv2.putText(img, label, (x1, y1 - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    # 画像の表示
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'RF-DETR-N Detection Results ({len(detections)} objects)')
    plt.show()
    
    return img

# 可視化
# result_img = visualize_detections(test_image_path, detections, CLASSES, threshold=0.5)


## **Step 4: トレーニング済みモデルの保存と再開**


In [ ]:
# トレーニングの再開（必要に応じて）
# 
# checkpoint_path = os.path.join(OUTPUT_DIR, 'checkpoint_epoch_50.pt')  # 例
# 
# model = RFDETRNano()
# model.train(
#     dataset_dir=COCO_DATA_DIR,
#     epochs=EPOCHS,
#     batch_size=BATCH_SIZE,
#     resume=checkpoint_path,
#     device="cuda"
# )


## **補足: RF-DETR モデルバリアント**

RF-DETRには以下のモデルバリアントがあります：

| Model | Parameters | COCO mAP | Speed |
|-------|------------|----------|-------|
| RF-DETR-N (Nano) | ~5M | ~40 | Fastest |
| RF-DETR-B (Base) | ~29M | ~53 | Fast |
| RF-DETR-L (Large) | ~128M | ~56 | Moderate |

他のバリアントを使用する場合：

```python
from rfdetr import RFDETRBase, RFDETRLarge

model = RFDETRBase()  # Base model
model = RFDETRLarge()  # Large model
```


In [ ]:
# モデルのエクスポート（ONNX形式）
# 
# export_path = os.path.join(OUTPUT_DIR, 'rf_detr_nano.onnx')
# model.export(export_path, format='onnx')
# print(f"Model exported to {export_path}")
